In [1]:
from autogen_tools import *
# 使用get_legal_document函数查找相关案件
case_num_1 = "(2020)赣0191民初1045号"
case_num_2 = "(2019)晋0402民初918号"
# 比较并打印结果
legal_document = get_legal_document(case_num=[case_num_1, case_num_2])
if float(legal_document[0]['涉案金额']) > float(legal_document[1]['涉案金额']):
    print(case_num_1 + "的涉案金额最大，为" + legal_document[0]['涉案金额'] + "元")
elif float(legal_document[1]['涉案金额']) > float(legal_document[0]['涉案金额']):
    print(case_num_2 + "的涉案金额最大，为" + legal_document[0]['涉案金额'] + "元")
else:
    print("两起案件涉案金额一样大，为" + legal_document[0]['涉案金额'] + "元")

(2019)晋0402民初918号的涉案金额最大，为56100元
